In [ ]:
#import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import numpy as np
import sqlite3

In [ ]:
#source of the tables
url='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

In [ ]:
#get page content
page= requests.get(url)
print(page.content)

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of S&amp;P 500 companies - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-conte

In [ ]:
#parse the html into a readable format
soup=BeautifulSoup(page.content,'html.parser')
print(soup.prettify())

In [ ]:
#make sure I was using the correct table
tables=soup.find_all('table',class_='sortable')
correct_table=tables[0]
print(correct_table.find_all('th'))

In [ ]:
#creating empty lists
symbol=[]
security=[]
sec_filings=[]
gics_sector=[]
gics_subindustry=[]
headquarters=[]
date_added=[]
cik=[]
founded=[]
rows= correct_table.find_all('tr')


In [ ]:
#for loop to loop through the table and append the respective content into the lists
for row in rows:
  cells=row.find_all('td')

  if(len(cells)==8):
    symbol.append(cells[0].get_text().rstrip('\n'))
    security.append(cells[1].get_text().rstrip('\n'))
    gics_sector.append(cells[2].get_text().rstrip('\n'))
    gics_subindustry.append(cells[3].get_text().rstrip('\n'))
    headquarters.append(cells[4].get_text().rstrip('\n'))
    date_added.append(cells[5].get_text().rstrip('\n'))
    cik.append(cells[6].get_text().rstrip('\n'))
    founded.append(cells[7].get_text().rstrip('\n'))
#used this to grab the ticker symbols for the api
comma_separated_symbols = ','.join(symbol)
print(comma_separated_symbols)

In [ ]:
#create a dataframe
df1 = pd.DataFrame({'Symbol':symbol,
                   'Security':security,
                   'Gics_sector':gics_sector,
                   'Gics_subindustry':gics_subindustry,
                   'Headquarters':headquarters,
                   'Date_added':date_added,
                   'CIK':cik,
                   'Founded':founded
                   })
df1

In [ ]:
#using the api according to documentation
url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":comma_separated_symbols

               }



headers = {
    'x-api-key': "WvdW167KdV14xoSrqsVPP2PHOByblmqS9NbME40j"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
#parsing and printing the json
data=response.json()
formatted_json=json.dumps(data,sort_keys=True,indent=1)
print(formatted_json)

{
 "quoteResponse": {
  "error": null,
  "result": [
   {
    "ask": 153.16,
    "askSize": 10,
    "averageAnalystRating": "2.2 - Buy",
    "averageDailyVolume10Day": 3380010,
    "averageDailyVolume3Month": 4272229,
    "bid": 145.0,
    "bidSize": 8,
    "bookValue": 8.295,
    "corporateActions": [
     {
      "header": "Dividend",
      "message": "MMM announced a cash dividend of 0.73 with an ex-date of May. 23, 2025",
      "meta": {
       "amount": "0.73",
       "dateEpochMs": 1747972800000,
       "eventType": "DIVIDEND"
      }
     }
    ],
    "cryptoTradeable": false,
    "currency": "USD",
    "customPriceAlertConfidence": "HIGH",
    "dividendDate": 1741737600,
    "dividendRate": 2.92,
    "dividendYield": 1.91,
    "earningsCallTimestampEnd": 1745326800,
    "earningsCallTimestampStart": 1745326800,
    "earningsTimestamp": 1745317806,
    "earningsTimestampEnd": 1753709400,
    "earningsTimestampStart": 1753363800,
    "epsCurrentYear": 7.66076,
    "epsForward": 7

In [ ]:
#creating lists for the api
symbol1=[]
price=[]
fiftytwoweeklow=[]
fiftytwoweekhigh=[]
fiftydayaverage=[]
fiftydayaveragechange=[]
#loop through the json and extract infomation to put into the lists
for x in data['quoteResponse']['result']:
  symbol1.append(x.get('symbol'))
  price.append(x.get('regularMarketPrice'))
  fiftytwoweeklow.append(x.get('fiftyTwoWeekLow'))
  fiftytwoweekhigh.append(x.get('fiftyTwoWeekHigh'))
  fiftydayaverage.append(x.get('fiftyDayAverage'))
  fiftydayaveragechange.append(x.get('fiftyDayAverageChange'))

In [ ]:
#creating the second dataframe
df2 = pd.DataFrame({'Symbol':symbol1,
                    'Price':price,
                    'Fifty two week low':fiftytwoweeklow,
                    'Fifty two week high':fiftytwoweekhigh,
                    'Fifty day average':fiftydayaverage,
                    'Fifty day average change':fiftydayaveragechange
                   }
)

df2

,Symbol,Price,Fifty two week low,Fifty two week high,Fifty day average,Fifty day average change
0,MMM,153.11,96.76,156.35,142.5096,10.600403
1,AOS,70.52,58.83,92.45,66.0834,4.436600
2,ABT,134.80,99.71,141.23,129.8662,4.933807
3,ABBV,184.02,153.58,218.66,193.7324,-9.712402
4,ACN,317.69,275.01,398.35,305.3876,12.302399
...,...,...,...,...,...,...
498,XYL,128.03,100.47,146.08,118.3426,9.687401
499,YUM,148.07,122.13,163.30,150.7114,-2.641388
500,ZBRA,300.62,205.73,427.76,262.6924,37.927580
501,ZBH,96.95,89.92,120.13,103.8386,-6.888603


In [ ]:
#doing an inner merge and showing the merged df
df3=pd.merge(df1,df2,how='inner',on='Symbol')
df3


,Symbol,Security,Gics_sector,Gics_subindustry,Headquarters,Date_added,CIK,Founded,Price,Fifty two week low,Fifty two week high,Fifty day average,Fifty day average change
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902,153.11,96.76,156.35,142.5096,10.600403
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916,70.52,58.83,92.45,66.0834,4.436600
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888,134.80,99.71,141.23,129.8662,4.933807
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888),184.02,153.58,218.66,193.7324,-9.712402
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989,317.69,275.01,398.35,305.3876,12.302399
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,0001524472,2011,128.03,100.47,146.08,118.3426,9.687401
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,0001041061,1997,148.07,122.13,163.30,150.7114,-2.641388
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,0000877212,1969,300.62,205.73,427.76,262.6924,37.927580
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,0001136869,1927,96.95,89.92,120.13,103.8386,-6.888603


In [ ]:
df3.describe()

,Price,Fifty two week low,Fifty two week high,Fifty day average,Fifty day average change
count,501.000000,501.000000,501.000000,501.000000,501.000000
mean,224.936377,167.347840,263.914290,211.025618,13.910759
std,479.423107,371.291796,575.743202,452.662815,38.870960
min,8.840000,6.640000,11.480000,8.507600,-176.213800
25%,69.320000,52.100000,84.670000,66.469900,0.371601
50%,125.900000,95.850000,154.320000,118.342600,5.276199
75%,237.960000,172.720000,273.890000,224.881800,15.633598
max,7370.410000,6562.850000,9964.770000,7195.336000,579.679200


In [ ]:
#export to CSV
df3.to_csv('Group1_Project1_Question2.csv',index=False)

In [ ]:
#CREATE DATABASE in sqlite
db_conn = sqlite3.connect('S&P500.db')

In [ ]:
cursor = db_conn.cursor() #db_conn will serve as a bridge between the db file and python
#.cursor() gives you a cursor, which is like a tool or pointer that lets you run SQL commands (like SELECT, INSERT, etc.) on the database.

In [ ]:
df3.to_sql('df3', db_conn, if_exists='replace', index=False)


503

In [ ]:
db_conn.close()
